# Analyze airline fleets

## The dataset was found at: https://www.kaggle.com/traceyvanp/airlinefleet

## Initial setup

In [173]:
# importing the necessary libraries

import pandas as pd
import numpy as np

In [174]:
# reading in the data

fleet = pd.read_csv('Fleet Data.csv')

In [175]:
# having a first look at the data

fleet.sample(5)

,Parent Airline,Airline,Aircraft Type,Current,Future,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
156,Air China,Air China Cargo,Tupolev Tu-204,NaN,NaN,1.0,1.0,4.0,$16,$0,NaN
1284,SAS Group,Scandinavian Airlines,ATR 42/72,13.0,NaN,7.0,20.0,NaN,$22,$287,2.0
976,Lion Air,Lion Air,McDonnell Douglas MD-80,NaN,NaN,20.0,20.0,NaN,$45,$0,NaN
934,Korean Air,Korean Air,Eurocopter EC135,5.0,NaN,NaN,5.0,NaN,NaN,$0,5.9
1582,Wizz Air,Wizz Air,Airbus A321,11.0,4.0,NaN,15.0,129.0,$115,"$1,264",0.6


## Data preparation and cleaning

In [176]:
fleet.describe()

,Current,Future,Historic,Total,Orders,Average Age
count,859.000000,188.000000,1113.000000,1484.000000,348.000000,820.000000
mean,24.033760,3.382979,14.513028,24.955526,26.419540,10.115000
std,41.091234,4.656331,23.763373,46.651526,43.024179,6.859362
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.100000
25%,5.000000,1.000000,3.000000,4.000000,5.000000,5.000000
50%,12.000000,2.000000,7.000000,11.000000,13.500000,8.900000
75%,26.500000,4.000000,16.000000,27.000000,28.250000,14.500000
max,718.000000,38.000000,325.000000,952.000000,400.000000,39.000000


In [177]:
fleet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583 entries, 0 to 1582
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Parent Airline        1583 non-null   object 
 1   Airline               1583 non-null   object 
 2   Aircraft Type         1583 non-null   object 
 3   Current               859 non-null    float64
 4   Future                188 non-null    float64
 5   Historic              1113 non-null   float64
 6   Total                 1484 non-null   float64
 7   Orders                348 non-null    float64
 8   Unit Cost             1548 non-null   object 
 9   Total Cost (Current)  1556 non-null   object 
 10  Average Age           820 non-null    float64
dtypes: float64(6), object(5)
memory usage: 136.2+ KB


### Apparently, there are lots of null values. But that's not a big problem

In [178]:
# function to count the infinite values

def get_inf_count(df):
    return {
        col : df[
            df[col].isin([np.inf, -np.inf])
        ].shape[0] for col in df.columns
    }

get_inf_count(fleet)

{'Parent Airline': 0,
 'Airline': 0,
 'Aircraft Type': 0,
 'Current': 0,
 'Future': 0,
 'Historic': 0,
 'Total': 0,
 'Orders': 0,
 'Unit Cost': 0,
 'Total Cost (Current)': 0,
 'Average Age': 0}

In [179]:
# checking for duplicates

fleet[fleet.duplicated()]

,Parent Airline,Airline,Aircraft Type,Current,Future,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
998,Lufthansa,Brussels Airlines,Airbus A319,21.0,1.0,1.0,23.0,2.0,$90,"$1,882",12.6
1008,Lufthansa,Brussels Airlines,Airbus A320,9.0,NaN,NaN,9.0,1.0,$98,$882,13.1
1018,Lufthansa,Brussels Airlines,Airbus A330,9.0,1.0,NaN,10.0,1.0,$240,"$2,160",19.3
1037,Lufthansa,Brussels Airlines,Boeing 737,NaN,NaN,11.0,11.0,NaN,$74,$0,NaN
1055,Lufthansa,Brussels Airlines,British Aerospace BAe 146/Avro RJ,8.0,NaN,25.0,33.0,NaN,$14,$109,18.3
1067,Lufthansa,Brussels Airlines,De Havilland Canada DHC-8 Dash 8,2.0,NaN,3.0,5.0,NaN,$31,$63,8.3
1071,Lufthansa,Brussels Airlines,Embraer ERJ-145,1.0,NaN,NaN,1.0,NaN,$20,$20,15.5


In [180]:
# drop duplicate rows

fleet = fleet.drop_duplicates()

## Exploratory Analysis - Comparing "Lufthansa" with "Qatar Airways"

In [181]:
# transform costs to float values to be able to calculate with them

#get rid of the dollar sign
fleet['Unit Cost'] = fleet['Unit Cost'].str[1:]
fleet['Total Cost (Current)'] = fleet['Total Cost (Current)'].str[1:]

#convert string to float
fleet['Unit Cost'] = pd.to_numeric(fleet['Unit Cost'], errors='coerce')
fleet['Total Cost (Current)'] = pd.to_numeric(fleet['Total Cost (Current)'], errors='coerce')

fleet.dtypes

Parent Airline           object
Airline                  object
Aircraft Type            object
Current                 float64
Future                  float64
Historic                float64
Total                   float64
Orders                  float64
Unit Cost               float64
Total Cost (Current)    float64
Average Age             float64
dtype: object

In [182]:
# make a lufthansa df

lufthansa = fleet[fleet['Parent Airline'] == 'Lufthansa']

lufthansa.head()

,Parent Airline,Airline,Aircraft Type,Current,Future,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
990,Lufthansa,Lufthansa,Airbus A300,NaN,NaN,26.0,26.0,NaN,75.0,0.0,NaN
991,Lufthansa,Lufthansa,Airbus A310,NaN,NaN,25.0,25.0,NaN,75.0,0.0,NaN
992,Lufthansa,Austrian Airlines,Airbus A310,NaN,NaN,4.0,4.0,NaN,75.0,0.0,NaN
993,Lufthansa,Lufthansa,Airbus A319,30.0,NaN,5.0,35.0,NaN,90.0,NaN,14.9
994,Lufthansa,Austrian Airlines,Airbus A319,7.0,NaN,NaN,7.0,NaN,90.0,627.0,12.0


In [183]:
# make a qatar df

qatar = fleet[fleet['Parent Airline'] == 'Qatar Airways']

qatar.head()

,Parent Airline,Airline,Aircraft Type,Current,Future,Historic,Total,Orders,Unit Cost,Total Cost (Current),Average Age
1213,Qatar Airways,Qatar Airways,Airbus A300,NaN,NaN,9.0,9.0,NaN,75.0,0.0,NaN
1214,Qatar Airways,Qatar Airways Cargo,Airbus A300,NaN,NaN,3.0,3.0,NaN,75.0,0.0,NaN
1215,Qatar Airways,Qatar Airways,Airbus A310,NaN,NaN,2.0,2.0,NaN,75.0,0.0,NaN
1216,Qatar Airways,Qatar Executive,Airbus A319,2.0,NaN,NaN,NaN,10.0,90.0,179.0,NaN
1217,Qatar Airways,Qatar Airways,Airbus A319,2.0,NaN,1.0,3.0,NaN,90.0,179.0,13.4


In [189]:
# compare the unit cost means

lh_unitcost_mean = lufthansa['Unit Cost'].mean()
qa_unitcost_mean = qatar['Unit Cost'].mean()

print('Lufthansa average Unit Costs:', lh_unitcost_mean)
print('Qatar Airways average Unit Costs:', qa_unitcost_mean)

Lufthansa average Unit Costs: 105.61290322580645
Qatar Airways average Unit Costs: 160.45833333333334


### Qatar Airways has significant higher unit costs.